# 读取数据并用stdbscan输出聚类结果

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from stdbscan import STDBSCAN

In [2]:
def changetime(x):#改变为时间戳模式
    time = str(x)
    newtime = time[:-4] + ':' +time[-4:-2] + ':' + time[-2:]
    newtime_ = datetime.strptime(newtime, '%H:%M:%S')
    return newtime_

In [4]:
df_raw = pd.read_csv('0720QueryResults.csv',sep=",",converters={'crt_time': changetime})
df_raw = df_raw[df_raw['city'] == '青岛市']
df_raw = df_raw[['longitude','latitude','crt_time']]

In [5]:
def plot_clusters(df, output_name):#聚类图像
    import matplotlib.pyplot as plt

    labels = df['cluster'].values
    X = df[['longitude', 'latitude']].values

    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.title('ST-DSCAN: #n of clusters {}'.format(len(unique_labels)))
    plt.show()
    # plt.savefig(output_name)

In [6]:
def test_time(df):
    df = df.sample(n=50000,random_state = 1)
#     df.sample(frac=0.5,axis=0)
    '''
    transfrom the lon and lat to x and y
    need to select the right epsg
    I don't the true epsg of sample, but get the same result by using 
    epsg:4326 and epsg:32635
    '''
    st_dbscan = STDBSCAN(col_lat='latitude', col_lon='longitude',
                         col_time='crt_time', spatial_threshold=10000,
                         temporal_threshold=180, min_neighbors=11)
    df = st_dbscan.projection(df, p1_str='epsg:4326', p2_str='epsg:32635')
#     df = st_dbscan
    result_t600 = st_dbscan.run(df)
    return result_t600

In [7]:
%time
df = pd.DataFrame(test_time(df_raw))
print(pd.value_counts(df['cluster']))

Wall time: 0 ns
            longitude      latitude            crt_time
696781   7.625598e+06  1.050342e+07 1900-01-01 09:25:17
751778   7.592606e+06  1.045309e+07 1900-01-01 17:59:46
1119674  7.533278e+06  1.042871e+07 1900-01-01 16:40:54
429255   7.476516e+06  1.051981e+07 1900-01-01 13:10:53
124846   7.606978e+06  1.052168e+07 1900-01-01 19:02:23
...               ...           ...                 ...
614474   7.584906e+06  1.050947e+07 1900-01-01 09:44:02
1366288  7.569676e+06  1.051498e+07 1900-01-01 08:51:38
489884   7.476685e+06  1.052052e+07 1900-01-01 10:20:01
1130168  7.498748e+06  1.043724e+07 1900-01-01 13:31:07
316291   7.622176e+06  1.050790e+07 1900-01-01 17:59:50

[50000 rows x 3 columns]


C:\Users\85185\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\Users\85185\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


-1     16642
 1     11966
 3      6935
 4      6217
 5      2522
       ...  
 91       12
 90       12
 62        9
 59        9
 80        8
Name: cluster, Length: 92, dtype: int64


In [9]:
df[df.cluster == 1]

,longitude,latitude,crt_time,cluster
696781,7.625598e+06,1.050342e+07,1900-01-01 09:25:17,1
124846,7.606978e+06,1.052168e+07,1900-01-01 19:02:23,1
1342146,7.564629e+06,1.051145e+07,1900-01-01 10:55:59,1
1372601,7.569511e+06,1.051613e+07,1900-01-01 10:24:09,1
662106,7.556553e+06,1.051837e+07,1900-01-01 17:36:07,1
...,...,...,...,...
678432,7.613997e+06,1.050866e+07,1900-01-01 17:07:52,1
371472,7.583107e+06,1.050735e+07,1900-01-01 09:54:23,1
614474,7.584906e+06,1.050947e+07,1900-01-01 09:44:02,1
1366288,7.569676e+06,1.051498e+07,1900-01-01 08:51:38,1


In [16]:
df['latitude'].max(axis=0)

10590232.720614651

In [2]:
plot_clusters(df,1)

NameError: name 'plot_clusters' is not defined